In [1]:
import fsspec
import xarray as xr
import numpy as np
import rioxarray
from pathlib import Path
import psutil
from tqdm import tqdm
import concurrent

In [2]:
fs = fsspec.filesystem('s3',anon=True)

## List all availabe tiles

In [3]:
# fs.ls('s3://copernicus-dem-30m/')
# fs.ls('s3://copernicus-dem-90m/')

## Open a single 30m DEM

In [4]:
url = 's3://copernicus-dem-30m/Copernicus_DSM_COG_10_N45_00_W116_00_DEM/Copernicus_DSM_COG_10_N45_00_W116_00_DEM.tif'

In [5]:
rioxarray.open_rasterio(fs.open(url,'rb'), chunks = 'auto')

<xarray.DataArray (band: 1, y: 3600, x: 3600)>
dask.array<open_rasterio-b5bed3590e344bd25a120b66cd65a6fc<this-array>, shape=(1, 3600, 3600), dtype=float32, chunksize=(1, 3600, 3600), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -116.0 -116.0 -116.0 ... -115.0 -115.0 -115.0
  * y            (y) float64 46.0 46.0 46.0 46.0 46.0 ... 45.0 45.0 45.0 45.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Point
    scale_factor:   1.0
    add_offset:     0.0

## Save a single 30m DEM to disk

In [6]:
# with fs.open(url,'rb') as f:
#     with open(Path(url).name, 'wb') as lf:
#         lf.write(f.read())

## Build 30m tile urls for WA lat lon bounds

In [7]:
lons = np.arange(116,125,1)
lats = np.arange(45,50,1)

base_url = 's3://copernicus-dem-30m/'
urls = []

for lon in lons:
    for lat in lats:
        
        base_folder = 'Copernicus_DSM_COG_10_N'+str(lat)+'_00_W'+str(lon).zfill(3)+'_00_DEM/'
        dem_file = 'Copernicus_DSM_COG_10_N'+str(lat)+'_00_W'+str(lon).zfill(3)+'_00_DEM.tif'
        urls.append(base_url + base_folder + dem_file)

## Lazy load as xarray dataset

In [8]:
import dask
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    n_workers=10,
    threads_per_worker=2,
)

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 20,Total memory: 64.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58483,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 64.00 GiB
Comm: tcp://127.0.0.1:58541,Total threads: 2
Dashboard: http://127.0.0.1:58547/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:58486,


In [9]:
@dask.delayed
def xr_read(url):
    da = rioxarray.open_rasterio(fs.open(url,'rb'), chunks = 'auto')
    return da

In [10]:
results = []
for url in urls:
    results.append(xr_read(url))
results = dask.compute(results)

In [11]:
ds = xr.combine_by_coords(results[0])

In [12]:
ds

<xarray.DataArray (band: 1, y: 18000, x: 32400)>
dask.array<concatenate, shape=(1, 18000, 32400), dtype=float32, chunksize=(1, 3600, 3600), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -124.0 -124.0 -124.0 ... -115.0 -115.0 -115.0
  * y            (y) float64 50.0 50.0 50.0 50.0 50.0 ... 45.0 45.0 45.0 45.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Point
    scale_factor:   1.0
    add_offset:     0.0